In [1]:
// Mes imports de librairies externes

USE {
    repositories {
        //google()
        // à exécuter en dehors du proxy sinon KO
        mavenCentral()
        //maven("https://artifactory.mycloud.intrabpce.fr/artifactory/g-android-maven-proxy/")
        //maven("https://artifactory.mycloud.intrabpce.fr/artifactory/maven-third-party-android-libs/")
    }
    dependencies {
        val ktor_version = "2.2.4"
        implementation("io.ktor:ktor-client-apache-jvm:$ktor_version")
        implementation("io.ktor:ktor-serialization-kotlinx-json:$ktor_version")
    }
}

In [ ]:

import io.ktor.client.*
import io.ktor.client.request.*
import io.ktor.client.statement.*
import kotlinx.coroutines.runBlocking

val client = HttpClient()
runBlocking {
    client.get("https://google.com").bodyAsText()
 }

In [6]:
import io.ktor.client.engine.apache.*
import io.ktor.client.*
import io.ktor.client.request.*
import io.ktor.client.statement.*
import io.ktor.http.*
import kotlinx.serialization.SerialName
import kotlinx.serialization.Serializable
import kotlinx.serialization.decodeFromString
import kotlinx.serialization.json.Json

@Serializable
data class SearchProductList(
    @SerialName("products") val products: List<ProductDetail>? = null
)

@Serializable
data class Product(
    @SerialName("code") val productId: String? = null,
    @SerialName("product") val productDetail: ProductDetail? = null,
)

@Serializable
data class ProductDetail(
    @SerialName("code") val code: String = "",
    @SerialName("product_name_fr") val label: String = "",
    @SerialName("ingredients_text") val ingredientsText: String = "",
    @SerialName("ingredients_text_fr") val ingredientsTextFr: String = "",
    @SerialName("generic_name_fr") val genericName: String = "",
    @SerialName("image_url") val imageUrl: String = "",
    @SerialName("nutrition_grade_fr") val nutriScore: String = "",
    @SerialName("ecoscore_grade") val ecoScore: String = "",
    @SerialName("ecoscore_score") val ecoScoreValue: String = ""
)

class NutriScoreApp() {
    val jsonFormatter = Json { ignoreUnknownKeys = true }

    suspend fun getProduct(productId: String): Product {
        val client = HttpClient()
        val getProductUrl = "https://fr.openfoodfacts.org/api/v0/produit/$productId.json"
        val productText = client.get(getProductUrl + FIELDS).bodyAsText()
        val product: Product = jsonFormatter.decodeFromString(productText)
        client.close()
        return product
    }

    suspend fun searchProducts(search: String): List<ProductDetail> {
        val client = HttpClient()
        val searchProductUrl = "https://ssl-api.openfoodfacts.org/cgi/search.pl?search_simple=1&amp;json=1&amp;action=process&amp;sort_by=unique_scans_n"
        val extraParams = "&amp;search_terms=${search.encodeURLPath()}&amp;page=1"
        val searchText = client.get(searchProductUrl + FIELDS + extraParams).bodyAsText()
        val products: List<ProductDetail> = jsonFormatter.decodeFromString(searchText)
        client.close()
        return products
    }

    companion object {
        const val FIELDS = "&amp;fields=selected_images,image_url,product_name,brands,quantity,code,nutrition_grade_fr,ecoscore_grade,product_name_fr,nova_groups,allergens_tags,additives_tags,ingredients_from_palm_oil_n,carbon-footprint-from-known-ingredients_product,carbon-footprint-from-meat-or-fish_product,nutriments"
    }
}

In [7]:
import kotlinx.coroutines.runBlocking

// Exemple de produit :
// Gâteaux au chocolat : 3175681134935
runBlocking {
    val nutriScoreApp = NutriScoreApp()
    nutriScoreApp.getProduct("3175681134935")
 }


Product(productId=3175681134935, productDetail=ProductDetail(code=3175681134935, label=Sablé saveur Citron Yuzu au Maltitol, ingredientsText=, ingredientsTextFr=, genericName=, imageUrl=https://images.openfoodfacts.org/images/products/317/568/113/4935/front_fr.73.400.jpg, nutriScore=a, ecoScore=c, ecoScoreValue=))

In [8]:
import kotlinx.coroutines.runBlocking

// Exemple de recherche :
runBlocking {
    val search = "pate à tartiner noisette"
    val nutriScoreApp = NutriScoreApp()
    nutriScoreApp.searchProducts(search)
 }

Expected start of the array '[', but had 'EOF' instead at path: $
JSON input: .....cts/333/072/023/7378/nutrition_fr.74.100.jpg"}}}}],"skip":0}
kotlinx.serialization.json.internal.JsonDecodingException: Expected start of the array '[', but had 'EOF' instead at path: $
JSON input: .....cts/333/072/023/7378/nutrition_fr.74.100.jpg"}}}}],"skip":0}
	at kotlinx.serialization.json.internal.JsonExceptionsKt.JsonDecodingException(JsonExceptions.kt:24)
	at kotlinx.serialization.json.internal.JsonExceptionsKt.JsonDecodingException(JsonExceptions.kt:32)
	at kotlinx.serialization.json.internal.AbstractJsonLexer.fail(AbstractJsonLexer.kt:530)
	at kotlinx.serialization.json.internal.AbstractJsonLexer.fail$default(AbstractJsonLexer.kt:528)
	at kotlinx.serialization.json.internal.AbstractJsonLexer.fail$kotlinx_serialization_json(AbstractJsonLexer.kt:224)
	at kotlinx.serialization.json.internal.AbstractJsonLexer.unexpectedToken(AbstractJsonLexer.kt:207)
	at kotlinx.serialization.json.internal.StringJso